In [1]:
from __future__ import print_function

from optparse import OptionParser
import json
import os

# import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import pickle
import random
import time

import h5py
import numpy as np
import pandas as pd
import pysam
import seaborn as sns

sns.set(style="ticks", font_scale=1.3)

import tensorflow as tf

from basenji import seqnn
from basenji import stream
from basenji import dna_io

2022-11-18 10:30:40.089404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-11-18 10:30:40.089441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa") 

background_seqs = []
with open("/project/fudenber_735/tensorflow_models/akita/v2/analysis/background_seqs.fa", "r") as f:
    for line in f.readlines():
        if ">" in line:
            continue
        background_seqs.append(dna_io.dna_1hot(line.strip()))

seq_coords_df = pd.read_csv("/home1/smaruj/akita_utils/bin/insert_virtual_flanks_experiment/test_3rows.tsv", sep="\t")

In [3]:
# seq_coords_df.columns

In [4]:
def get_seq_back(ohe_sequence):
    ACTG_seq = str()
    for pos in range(len(ohe_sequence)):
        ACTG_seq = ACTG_seq + dna_io.hot1_get(ohe_sequence, pos)
    return ACTG_seq

In [5]:
def _insert_casette(seq_1hot, seq_1hot_insertion, spacer_bp, orientation_string):
        
    seq_length = seq_1hot.shape[0]
    insert_bp = len(seq_1hot_insertion)
    num_inserts = len(orientation_string)

    insert_plus_spacer_bp = insert_bp + 2 * spacer_bp
    multi_insert_bp = num_inserts * insert_plus_spacer_bp
    insert_start_bp = seq_length // 2 - multi_insert_bp // 2
    
    output_seq = seq_1hot.copy()
    insertion_starting_positions = []
    for i in range(num_inserts):
        offset = insert_start_bp + i * insert_plus_spacer_bp + spacer_bp

        insertion_starting_positions.append(offset)

        for orientation_arrow in orientation_string[i]:
            if orientation_arrow == ">":
                output_seq[offset : offset + insert_bp] = seq_1hot_insertion
            else:
                output_seq[offset : offset + insert_bp] = dna_io.hot1_rc(seq_1hot_insertion)

    return output_seq

In [6]:
# insertion experiments utils
def symmertic_insertion_seqs_gen(seq_coords_df, background_seqs, genome_open):
    """ sequence generator for making insertions from tsvs
        construct an iterator that yields a one-hot encoded sequence
        that can be used as input to akita via PredStreamGen
    """
        
    list_seq_1hot = []
    
    for s in seq_coords_df.itertuples():
        
        flank_bp = s.flank_bp
        spacer_bp = s.spacer_bp
        orientation_string = s.orientation
                
        seq_1hot_insertion = dna_io.dna_1hot(
            genome_open.fetch(s.chrom, s.start - flank_bp, s.end + flank_bp).upper()
        )
        
        if s.strand == "-":
            seq_1hot_insertion = dna_io.hot1_rc(seq_1hot_insertion)
            # now, all motifs are standarized to this orientation ">"

        seq_1hot = background_seqs[s.background_index].copy()
        seq_1hot = _insert_casette(seq_1hot, seq_1hot_insertion, spacer_bp, orientation_string)
        
        yield seq_1hot

In [ ]:
# i = 0

# for s in seq_coords_df.itertuples():
#     print("experiment_id: ", s.experiment_id)
    
#     ohe_CTCF = dna_io.dna_1hot(genome_open.fetch(s.chrom, s.start - s.flank_bp, s.end + s.flank_bp).upper())
#     print(get_seq_back(ohe_CTCF))
    
#     seq_length = 1310720
#     insert_bp = len(ohe_CTCF)
#     num_inserts = len(s.orientation)

#     insert_plus_spacer_bp = insert_bp + 2 * s.spacer_bp
#     multi_insert_bp = num_inserts * insert_plus_spacer_bp
#     insert_start_bp = seq_length // 2 - multi_insert_bp // 2
    
#     t_insertion_start = insert_start_bp + insert_plus_spacer_bp + s.spacer_bp
#     print(t_insertion_start)
#     print(get_seq_back(inserted_sequences[i][t_insertion_start : t_insertion_start + 5]))
    
#     i += 1

In [ ]:
# _insert_casette().. 
        #genome= GAAGG , background = GAAGGTTT, test: _insert_casette( utils.dna_1hot(insertino), utils.dna_1hot(background), ints & strings)

In [ ]:
'experiment_id', 'chrom', 'start', 'end', 'strand', 'genomic_SCD',
       'orientation', 'background_index', 'flank_bp', 'spacer_bp'

In [ ]:
d = {"experiment_id": [1, 2], "start": [2, 8], "end": [5, 12], "orientation": [">", ">"], "background_index": [0, 0], "flank_bp": [1, 0], "spacer_bp": [0,0]}
df = pd.DataFrame(data=d)
df

In [17]:
toy_genome_open = dna_io.dna_1hot("TTGTACTTCGTCGTT")
seq_1hot = dna_io.dna_1hot("AAAAAAAAAA")

In [18]:
first_spacer_bp = 1
first_seq_1hot_insertion = toy_genome_open[(2-first_spacer_bp):(6+first_spacer_bp)]
first_orientation_string = ">"

In [19]:
get_seq_back(_insert_casette(seq_1hot, first_seq_1hot_insertion, first_spacer_bp, first_orientation_string)) == "AATGTACTAA"

True

In [20]:
get_seq_back(_insert_casette(seq_1hot, first_seq_1hot_insertion, first_spacer_bp, first_orientation_string))

'AATGTACTAA'

In [21]:
second_spacer_bp = 0
second_seq_1hot_insertion = toy_genome_open[(8-second_spacer_bp):(13+second_spacer_bp)]
second_orientation_string = "<"

In [22]:
get_seq_back(second_seq_1hot_insertion)

'CGTCG'

In [23]:
get_seq_back(seq_1hot)

'AAAAAAAAAA'

In [26]:
get_seq_back(_insert_casette(seq_1hot, second_seq_1hot_insertion, second_spacer_bp, second_orientation_string)) == "AAACGACGAA"

True

In [27]:
get_seq_back(_insert_casette(seq_1hot, second_seq_1hot_insertion, second_spacer_bp, second_orientation_string))

'AAACGACGAA'